In [2]:
import json
import os

In [3]:
class Oven:
    def __init__(self, start_temp : int, working_temps : list[int], operations : list[str]) -> None:
        self.start_temp = start_temp
        self.working_temps = working_temps
        self.operations = operations

class Operation:
    def __init__(self, name : str, timing : int) -> None:
        self.name = name
        self.timing = timing

class Series:
    def __init__(self, temperature : int, operations : list[Operation]) -> None:
        self.temperature = temperature
        self.operations = [Operation(**json.loads(json.dumps(op))) for op in operations]

class DayData:
    def __init__(self, ovens : list[Oven], series : list[Series]) -> None:
        self.ovens = [Oven(**json.loads(json.dumps(oven))) for oven in ovens]
        self.series = [Series(**json.loads(json.dumps(ser))) for ser in series]

In [4]:
from enum import Enum
class OPERS(Enum):
    prokat = -1
    kovka = -2
    otzhig = -3
    nagrev = -4

In [16]:
dataset : list[DayData] = []
root = './train_dataset_train/train/'
# for _,_,files in os.walk(root):
#     for file in files:
#         dataset.append(DayData(**json.loads(open(root + file).read())))
path = 'mini-day1.json'
dataset.append(DayData(**json.loads(open(path).read())))


In [17]:
needed_temperature = []
needed_operations = {}
cnt_series = 1
cnt_number_operation = 1
for i in range(len(dataset[0].series)):
    all_series = dataset[0].series[i]
    needed_temperature.append(all_series.temperature)
    for j in range(len(all_series.operations)):
        all_operations = all_series.operations
        needed_operations[all_operations[j].name + '_' + str(cnt_series) + '_' + str(cnt_number_operation)] = [all_operations[j].timing, all_series.temperature]
        #первая цифра - номер серии; вторая цифра - номер операции в серии.
        cnt_number_operation += 1
    cnt_number_operation = 1
    cnt_series += 1
needed_operations


{'nagrev_1_1': [299, 960],
 'otzhig_1_2': [15, 960],
 'nagrev_2_1': [296, 980],
 'kovka_2_2': [10, 980],
 'kovka_2_3': [10, 980],
 'kovka_2_4': [10, 980],
 'nagrev_3_1': [300, 1020],
 'prokat_3_2': [15, 1020],
 'nagrev_4_1': [125, 990],
 'prokat_4_2': [15, 990],
 'nagrev_5_1': [249, 1040],
 'kovka_5_2': [20, 1040],
 'kovka_5_3': [10, 1040],
 'nagrev_6_1': [268, 1020],
 'otzhig_6_2': [15, 1020],
 'nagrev_7_1': [222, 980],
 'prokat_7_2': [15, 980],
 'nagrev_8_1': [221, 1020],
 'otzhig_8_2': [15, 1020],
 'nagrev_9_1': [71, 1070],
 'otzhig_9_2': [15, 1070],
 'nagrev_10_1': [232, 1000],
 'kovka_10_2': [10, 1000],
 'kovka_10_3': [10, 1000],
 'kovka_10_4': [20, 1000],
 'nagrev_11_1': [82, 980],
 'prokat_11_2': [15, 980],
 'nagrev_12_1': [213, 1040],
 'prokat_12_2': [15, 1040],
 'nagrev_13_1': [76, 1080],
 'kovka_13_2': [15, 1080],
 'nagrev_14_1': [96, 980],
 'otzhig_14_2': [15, 980],
 'nagrev_15_1': [245, 1080],
 'prokat_15_2': [15, 1080],
 'nagrev_16_1': [62, 970],
 'otzhig_16_2': [15, 970],

In [ ]:
for_timings = list(needed_operations.items())
for i in range(len(for_timings) - 1):
    cur_pair = for_timings[i]
    next_pair = for_timings[i + 1]
    if 'kovka' in cur_pair[0] and 'kovka' in next_pair[0]:
        cur_elems = cur_pair[0].split('_')
        next_elems = next_pair[0].split('_')
        if int(cur_elems[1]) == int(next_elems[1]):
            needed_operations['podogrev_' + str(cur_elems[1]) + '_' + str(cur_elems[2])] = [120, 0]
needed_operations


In [8]:
ovens_temperature = []
for i in range(len(dataset[0].ovens)):
    all_series = dataset[0].ovens[i]
    ovens_temperature.append(all_series.start_temp)
ovens_temperature

[980, 1010, 1100, 1030, 1130, 1090, 1020]

In [24]:
operations : dict[int : list[int]]= {} ### список вида [температура печи: [номер серии, ...номер серии]]
times = {}
opers = []
false_opers = []
for i in range(len(ovens_temperature)):
    start_t = ovens_temperature[i]
    times[i] = 0
    operations[start_t] = []
    for j in range(len(needed_temperature)):
        if start_t == needed_temperature[j]:
            operations[start_t].append(j)
            opers.append(j)
for i in range(len(needed_temperature)):
    if i not in opers:
        false_opers.append(i)
opers.sort()
# opers

[1, 2, 5, 6, 7, 10, 13, 16, 19, 23, 24, 25, 26]

In [10]:
operations

{980: [1, 6, 10, 13],
 1010: [23],
 1100: [],
 1030: [16, 19],
 1130: [],
 1090: [24, 25, 26],
 1020: [2, 5, 7]}

In [11]:
ser_length : int = dict()   ###Общее время работы на одну серию
pr_ser = '1'
for key, val in needed_operations.items():
    now = key.split('_')[1]
    try:
        ser_length[f'Series_sum_{now}'] : int = ser_length[f'Series_sum_{now}'] + val[0]
    except:
        # continue
        ser_length[f'Series_sum_{now}'] : int = val[0]
    # if now == pr_ser:
        # ser_length[f'Series_sum_{now}'] : int =+ val
    # pr_ser = key.split('_')[1]
ser_length
# gen_time = float(f'{sum(ser_length.values()) / 60 / 7:.3f}')    ### общее количество часов работы на одну печь + ковка/прокат
# gen_time

{'Series_sum_1': 314,
 'Series_sum_2': 566,
 'Series_sum_3': 315,
 'Series_sum_4': 140,
 'Series_sum_5': 399,
 'Series_sum_6': 283,
 'Series_sum_7': 237,
 'Series_sum_8': 236,
 'Series_sum_9': 86,
 'Series_sum_10': 512,
 'Series_sum_11': 97,
 'Series_sum_12': 228,
 'Series_sum_13': 91,
 'Series_sum_14': 111,
 'Series_sum_15': 260,
 'Series_sum_16': 77,
 'Series_sum_17': 60,
 'Series_sum_18': 102,
 'Series_sum_19': 111,
 'Series_sum_20': 492,
 'Series_sum_21': 295,
 'Series_sum_22': 186,
 'Series_sum_23': 138,
 'Series_sum_24': 294,
 'Series_sum_25': 174,
 'Series_sum_26': 87,
 'Series_sum_27': 122}

In [51]:
ops = dict()
oven_pars : list[list[str], int, list[int]] = []
for i in dataset[0].ovens:
    stt = i.start_temp
    wtmp = i.working_temps
    ops = i.operations
    ops = [[tp.value for tp in OPERS if tp.name == d][0] for d in ops]
    oven_pars.append([list(dict().fromkeys(ops)), stt, wtmp])
oven_pars


[[[-1], 980, [1040, 970, 1180, 1030, 980]],
 [[-2], 1010, [960, 1180, 1110, 1010]],
 [[-1], 1100, [1060, 1160, 1070, 1110, 950, 990, 1100]],
 [[-2], 1030, [1220, 1160, 1000, 1230, 1200, 1240, 1030]],
 [[-1], 1130, [1080, 1130, 1100]],
 [[-1], 1090, [1090, 1220, 1000, 1130, 1100, 1200, 1010, 1110, 1150]],
 [[-3], 1020, [960, 970, 1070, 1010, 1170, 1020, 990]]]

In [43]:
# ser_pars = [[[(op.name, op.timing) for op in i.operations], i.temperature] for i in dataset[0].series]
ser_pars = [[[([tp.value for tp in OPERS if tp.name == op.name][0], op.timing) for op in i.operations], i.temperature] for i in dataset[0].series]
train_dataset = []
train_dataset.extend(ser_pars)
train_dataset.extend(oven_pars)
# ser_pars + oven_pars 
sample = ser_pars[0][0][0][0]
# for i in OPERS:
    # print(i.value if i.name == sample else None)
ser_pars

[[[(-4, 299), (-3, 15)], 960],
 [[(-4, 296), (-2, 10), (-2, 10), (-2, 10)], 980],
 [[(-4, 300), (-1, 15)], 1020],
 [[(-4, 125), (-1, 15)], 990],
 [[(-4, 249), (-2, 20), (-2, 10)], 1040],
 [[(-4, 268), (-3, 15)], 1020],
 [[(-4, 222), (-1, 15)], 980],
 [[(-4, 221), (-3, 15)], 1020],
 [[(-4, 71), (-3, 15)], 1070],
 [[(-4, 232), (-2, 10), (-2, 10), (-2, 20)], 1000],
 [[(-4, 82), (-1, 15)], 980],
 [[(-4, 213), (-1, 15)], 1040],
 [[(-4, 76), (-2, 15)], 1080],
 [[(-4, 96), (-3, 15)], 980],
 [[(-4, 245), (-1, 15)], 1080],
 [[(-4, 62), (-3, 15)], 970],
 [[(-4, 45), (-1, 15)], 1030],
 [[(-4, 87), (-1, 15)], 950],
 [[(-4, 96), (-3, 15)], 960],
 [[(-4, 207), (-2, 20), (-2, 15), (-2, 10)], 1030],
 [[(-4, 135), (-2, 20), (-2, 20)], 1060],
 [[(-4, 171), (-1, 15)], 1050],
 [[(-4, 123), (-1, 15)], 1060],
 [[(-4, 139), (-2, 20), (-2, 15)], 1010],
 [[(-4, 154), (-2, 20)], 1090],
 [[(-4, 72), (-1, 15)], 1090],
 [[(-4, 107), (-3, 15)], 1090]]

In [49]:
CHAINED_SERIES = []
for i in ser_pars:
    for j in i:
        try:
            for k in j:
                for _ in k:
                    CHAINED_SERIES.append(_)
                    # print(_, 'deep')
        except:
            CHAINED_SERIES.append(j)

            # print(j)

CHAINED_SERIES[:10]
# CHAINED_SERIES.__len__() # 153


[-4, 299, -3, 15, 960, -4, 296, -2, 10, -2]

In [52]:
CHAINED_OVENS = []
for i in oven_pars:
    for j in i:
        try:
            for k in j:
                try:
                    for _ in k:
                        CHAINED_OVENS.append(_)
                    # print(_, 'deep')
                except TypeError:
                    CHAINED_OVENS.append(k)
        except:
            CHAINED_OVENS.append(j)

                # print(j)

CHAINED_OVENS
# CHAINED_OVENS.__len__() # 56


[-1,
 980,
 1040,
 970,
 1180,
 1030,
 980,
 -2,
 1010,
 960,
 1180,
 1110,
 1010,
 -1,
 1100,
 1060,
 1160,
 1070,
 1110,
 950,
 990,
 1100,
 -2,
 1030,
 1220,
 1160,
 1000,
 1230,
 1200,
 1240,
 1030,
 -1,
 1130,
 1080,
 1130,
 1100,
 -1,
 1090,
 1090,
 1220,
 1000,
 1130,
 1100,
 1200,
 1010,
 1110,
 1150,
 -3,
 1020,
 960,
 970,
 1070,
 1010,
 1170,
 1020,
 990]

In [53]:
SAMPLE_LENGHT_OVENS = 80
SAMPLE_LENGHT_SERIES = 180
SAMPLE = CHAINED_OVENS + [0] * SAMPLE_LENGHT_OVENS + CHAINED_SERIES + [0] * SAMPLE_LENGHT_SERIES

In [54]:
import numpy as np
SAMPLE = np.array(SAMPLE)
# SAMPLE = SAMPLE / max(SAMPLE)
SAMPLE

array([  -1,  980, 1040,  970, 1180, 1030,  980,   -2, 1010,  960, 1180,
       1110, 1010,   -1, 1100, 1060, 1160, 1070, 1110,  950,  990, 1100,
         -2, 1030, 1220, 1160, 1000, 1230, 1200, 1240, 1030,   -1, 1130,
       1080, 1130, 1100,   -1, 1090, 1090, 1220, 1000, 1130, 1100, 1200,
       1010, 1110, 1150,   -3, 1020,  960,  970, 1070, 1010, 1170, 1020,
        990,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,   -4,  299,   -3,   15,  960,   -4,  296,
         -2,   10,   -2,   10,   -2,   10,  980,   

In [55]:
#Синтетические данные
import torch
SAMPLE_LENGHT_OVENS = 80
SAMPLE_LENGHT_SERIES = 180
SAMPLE = [CHAINED_OVENS + [0] * SAMPLE_LENGHT_OVENS + CHAINED_SERIES + [0] * SAMPLE_LENGHT_SERIES] * 100
SAMPLE = torch.tensor(np.array(SAMPLE), dtype=torch.float32)
# SAMPLE = SAMPLE / max(SAMPLE)
# SAMPLE.shape   #torch.Size([100, 469])


In [83]:
LABEL = np.load('labls.npy')
LABEL.shape
## Полусинтетические данные нужного формата
#Не успели обработать все файлы
LABEL = [LABEL] * 100
LABEL[-10][::-13]


array([   0, 1138,  504, 1020,    0,   87, 1090,    0,  213,    0, 1011,
        362, 1160,    0,  303, 1100,    0,  776,    0,    0,  334, 1040,
       1384,  735, 1170,    0,  467, 1100,    0,  555,   -1,    0,  492,
       1030,    0,  683,  950,    0,  906,  279,   -2,    0,    0,    0,
        956,  268,   -3,    0,    0,   -1,    0, 1130,    0, 1001,  227,
          0,    0,  243,   -1,    0,  646, 1180,    0,  319,    0, 1273,
        519,  970,    0,  332, 1220,    0,  348,    0, 1207,  482, 1230,
          0,  438, 1160,    0,  786,  139,    0,  754, 1180, 1399,  750,
        990,    0,  758, 1010,    0,    0,    0,    0,  861,  207,    0,
          0, 1100,    0,    0,  294,    0,    0,  222, 1384,  750, 1020,
        467, 1100,  348, 1087,  362, 1000,  303, 1100,  646, 1180,  237,
         -1,  821,    0,  758, 1010,  435, 1222,  482, 1200,  438, 1160,
        656, 1010,  319,    0,  956,  268,   -3, 1150,  570,    0,  492,
       1030,  563, 1110,  776,  139,  334, 1040, 10

In [64]:
import torch
LABEL = torch.tensor(LABEL, dtype=torch.float32)
LABEL

tensor([[-1.,  0.,  0.,  ...,  0.,  0.,  0.],
        [-1.,  0.,  0.,  ...,  0.,  0.,  0.],
        [-1.,  0.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [-1.,  0.,  0.,  ...,  0.,  0.,  0.],
        [-1.,  0.,  0.,  ...,  0.,  0.,  0.],
        [-1.,  0.,  0.,  ...,  0.,  0.,  0.]])

In [98]:
from sklearn.preprocessing import normalize 
from sklearn import tree
X = SAMPLE  # массив данных на которых обучается нейросеть
y = LABEL
clf = tree.DecisionTreeRegressor()
clf = clf.fit(X, y)
# clf.predict([[1, 1]])
a = clf.predict(X[0].reshape(1, -1))
print('Score is ',clf.score(X, y))
for i in a[0]:
    print(i)    

Score is  1.0
-1.0
0.0
0.0
980.0
1040.0
970.0
1180.0
1030.0
980.0
0.0
222.0
222.0
237.0
237.0
319.0
319.0
334.0
334.0
634.0
754.0
769.0
-2.0
0.0
0.0
1010.0
960.0
1180.0
1110.0
1010.0
0.0
139.0
139.0
159.0
159.0
279.0
279.0
294.0
294.0
526.0
646.0
656.0
656.0
776.0
776.0
786.0
786.0
906.0
906.0
926.0
-1.0
0.0
0.0
1100.0
1060.0
1160.0
1070.0
1110.0
950.0
990.0
1100.0
0.0
123.0
243.0
258.0
258.0
303.0
423.0
438.0
438.0
563.0
683.0
698.0
-2.0
0.0
0.0
1030.0
1220.0
1160.0
1000.0
1230.0
1200.0
1240.0
1030.0
0.0
207.0
207.0
227.0
227.0
347.0
347.0
362.0
362.0
482.0
482.0
492.0
492.0
741.0
861.0
881.0
881.0
1001.0
1001.0
1011.0
1011.0
1087.0
1207.0
1222.0
-1.0
0.0
0.0
1130.0
1080.0
1130.0
1100.0
0.0
213.0
333.0
348.0
348.0
435.0
555.0
570.0
-1.0
0.0
0.0
1090.0
1090.0
1220.0
1000.0
1130.0
1100.0
1200.0
1010.0
1110.0
1150.0
0.0
72.0
72.0
87.0
87.0
332.0
452.0
467.0
467.0
638.0
758.0
773.0
-3.0
0.0
0.0
1020.0
960.0
970.0
1070.0
1010.0
1170.0
1020.0
990.0
0.0
268.0
268.0
283.0
283.0
504.0
504.0
51